In [1]:
# Import the packages - these are already contained in the pre-built environment. No (re) installation required. 
import xarray as xr
import pandas as pd
import os
import matplotlib.pyplot as plt
import datetime as dt



ModuleNotFoundError: No module named 'xarray'

In [2]:
# Read the sensor mapping from the CSV file using pandas
#
#This is the actual csv data, as returned from the database query
meas = pd.read_csv('../data/my_measurements.csv')


#Read in ancillary files: 
#
#This csv table, maps the sensor id to a sensor name
sensor_mapping_df = pd.read_csv('../data/table_Sensors.csv')

#This csv table, maps the station id to a latitude and a longituide
metgeo = pd.read_csv('../data/Location_File.csv')




NameError: name 'pd' is not defined

In [ ]:
# Add an "aware" time-valued column
meas['time'] = pd.to_datetime(meas['dt'])
## Show the time (no longer a string...)
print(meas['time'])

In [ ]:
# Create a dictionary mapping sensorID to sensor names
names = dict(zip(sensor_mapping_df['SensorID'], sensor_mapping_df['symbol']))

# Replace the sensorID integers with their corresponding names
meas['name'] = meas['sensorid'].replace(names)

In [ ]:
#Reorganize the data, so to "stack it by sensorid" 
meas_pivoted = meas.pivot(index='time', columns='name', values='value')

print(meas_pivoted)

In [ ]:
#COnvert to xarray - it is easy! 
ds = meas_pivoted.to_xarray()
print(ds)

In [ ]:
#The dataset, as well as the variables containing it, still have no metadata (metainformnation), describing the product. 
# Let us start by creating some dictionaries to map each variable to the metadata we need. 

# input is always "symbol" (i.e. the quantity name) and output is either units, the complete long name, or the decimal used to describe it

mapUnits = dict(zip(sensor_mapping_df['symbol'], sensor_mapping_df['unit']))
mapLongName = dict(zip(sensor_mapping_df['symbol'], sensor_mapping_df['en_name']))
mapDecimals = dict(zip(sensor_mapping_df['symbol'], sensor_mapping_df['decimal_num']))
loc = dict(zip(metgeo['stNo'], metgeo['station_nm']))

In [ ]:
#Get lat and lon of the station: 
_lat = dict(zip(metgeo['stNo'], metgeo['latt']))
_lon = dict(zip(metgeo['stNo'], metgeo['long']))

In [ ]:
#Now, for each variable, append this information
for varname in ds: 
    #print(varname, mapUnits[varname], mapLongName[varname])
    ds[varname].attrs['units'] = mapUnits[varname]
    ds[varname].attrs['long_name'] = mapLongName[varname]
    ds[varname].attrs['decimals'] = mapDecimals[varname]

In [ ]:
#Same goes for the whole dataset (i.e. the infos about the station)   
ds.attrs['station_name'] = loc[meas['stno'].unique()[0]]
ds.attrs['lat'] = _lat[meas['stno'].unique()[0]]
ds.attrs['lon'] = _lon[meas['stno'].unique()[0]]
ds.attrs['date_created'] = str(dt.datetime.now())
try: 
    ds.attrs['created by'] =  "{}, on {}".format(os.getlogin(), os.name) # POSIX stands for "Portable Operating System Interface for Unix."
except:
    pass
ds.attrs['xarray version'] = xr.__version__

In [ ]:
#Print the new, improved, updated version of your measurement
print(ds)

In [ ]:
#Save to netcdf
ds.to_netcdf('../output/my_measurements.nc', unlimited_dims='time')